In [1]:
import os
import cv2
import torch
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

device = "cuda:0"

processor = Blip2Processor.from_pretrained(
    os.path.join(os.environ["SCRATCH"], "mq_libs/blip2")
)
model = Blip2ForConditionalGeneration.from_pretrained(
    os.path.join(os.environ["SCRATCH"], "mq_libs/blip2"),
    torch_dtype=torch.float16,
)
model.to(device)


2023-10-31 07:24:31.445188: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-31 07:24:46.744261: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 07:24:46.751249: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 07:24:46.787543: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 07:24:58.235385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

In [ ]:
clip_id = "0076e425-bdb6-48b3-b4d3-695089ac9800"

with open(os.path.join(os.environ["SCRATCH"], "ego4d_data/v2/analysis_data/ground_truth_labels/ground_truth_labels.pickle"), "rb") as reader:
    ground_truth_labels = pickle.load(reader)[clip_id]

cap = cv2.VideoCapture(os.path.join(os.environ["SCRATCH"], "ego4d_data/v2/clips", clip_id + ".mp4"))




In [ ]:
questions = [
    "What does the image describe?",
    "What is the person in this picture doing?",
    "What is happening in this picture?",
    "This picture was taken with a camera mounted on the head of a person. What is this person doing?"
]
question_id = 3

frame_id = 7000
question = questions[question_id]

cap = cv2.VideoCapture(os.path.join(os.environ["SCRATCH"], "ego4d_data/v2/clips", clip_id + ".mp4"))
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id - 1)
_, frame = cap.read()

frame = Image.fromarray(frame[:, :, ::-1])

with torch.no_grad():
    all_generated_texts = []
    preprocessed_frames_batch_dict = processor(
        images=[
            frame
        ],
        text=["Question: " + question + " Answer:"],
        return_tensors="pt",
    ).to(device, torch.float16)
    generated_ids = model.generate(**preprocessed_frames_batch_dict)
    generated_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True
    )[0].strip()

display(frame)
print(generated_text)